In [ ]:
import asyncio
from copra.websocket import Channel, Client
import matplotlib.pyplot as plt
from collections import OrderedDict
from time import sleep
from dateutil import parser
import copy
import datetime
import itertools
from operator import itemgetter
import numpy as np
from dateutil.tz import tzutc
import math
import pytz
from pytz import timezone
import pickle
import json

In [ ]:
class Level2(Client):
    global file
    num_updates = 0
      
    # Receives message from API websocket
    def on_message(self, message):
        # Get snapshot of LOB from API and build internal representation
        if message['type'] == 'snapshot':
            self.starting_time = datetime.datetime.now(datetime.timezone.utc)
            print('Starting time: {}'.format(self.starting_time))
            print()
            json.dump([str(datetime.datetime.now(datetime.timezone.utc)), message['bids'], message['asks']],file)
            file.write("\n")

            
        # Update order book when new event occurs
        if message['type'] == 'l2update' and 'time' in message:
            for (side, price, amount) in message['changes']:
                json.dump({
                    "side": side, \
                    "price": price, \
                    "amount": amount, \
                    "time": message['time']      
                },file)
                file.write("\n")
                self.num_updates += 1
                if self.num_updates % 1000 == 0:
                    print("Number of updates: {}".format(self.num_updates))
                    print("Most Recent Update: {}".format((side, price, amount)))
                    print("Time: {}".format(message['time']))
                    print()
                    
    def on_error(message, reason):
        super().on_error(message,reason)
        print("Error: {}".format(message))
            

file = open("12_29_18.json","w")
loop = asyncio.get_event_loop()
channel = Channel('level2', 'ETC-USD')
ws = Level2(loop, channel)

async def my_task(seconds):
    global loop
    print('Collecting data for {} seconds'.format(seconds))
    await asyncio.sleep(seconds)
    await ws.close()
    return "Finished Task"

try:
    task_obj = loop.create_task(my_task(seconds=86400))
    loop.run_until_complete(task_obj)
except Exception as e:
    print(e)
finally:
    loop.close()
    file.close()
    print("Finished collecting data")

Starting time: 2018-12-29 04:34:37.440268+00:00

Number of updates: 1000
Most Recent Update: ('sell', '5.34000000', '5376.50053029')
Time: 2018-12-29T04:39:55.030Z

Number of updates: 2000
Most Recent Update: ('buy', '5.26000000', '3575.62692015')
Time: 2018-12-29T04:45:03.836Z

Number of updates: 3000
Most Recent Update: ('buy', '5.29000000', '1653.62735489')
Time: 2018-12-29T04:50:00.920Z

Number of updates: 4000
Most Recent Update: ('sell', '5.33000000', '201.4189722')
Time: 2018-12-29T04:53:35.746Z

Number of updates: 5000
Most Recent Update: ('buy', '5.30000000', '1870.27735489')
Time: 2018-12-29T04:59:24.181Z

Number of updates: 6000
Most Recent Update: ('buy', '5.29000000', '971.73735489')
Time: 2018-12-29T05:09:54.034Z

Number of updates: 7000
Most Recent Update: ('sell', '5.32000000', '11.2')
Time: 2018-12-29T05:16:02.154Z

Number of updates: 8000
Most Recent Update: ('sell', '5.33000000', '2238.58724203')
Time: 2018-12-29T05:23:54.629Z

Number of updates: 9000
Most Recent Upd

In [ ]:
def parse_file_json(file_name):
    with open(file_name, "r") as f:
        data = [json.loads(line) for line in f]
    starting_time = parser.parse(data[0][0])
    starting_bids = data[0][1]
    starting_asks = data[0][2]
    
    raw_updates = data[1:]
    
    Bids = {}
    Asks = {}    
    
    updates = []
    
    for price, amount in starting_bids:
        Bids[int(round((float(price)*100)))] = float(amount)
    for price, amount in starting_asks:
        Asks[int(round((float(price)*100)))] = float(amount)
        
    for u in raw_updates:
        price = int(round((float(u["price"])*100)))  
        side = u["side"]
        amount = float(u["amount"])
        time = parser.parse(u["time"])
        
        if side == "buy":
            change = amount - Bids.get(price,0)
        else:
            change = amount - Asks.get(price,0)
        
        updates.append({\
            "Bids": copy.copy(Bids), \
            "Asks": copy.copy(Asks), \
            "time": time, \
            "side": side, \
            "price": price, \
            "change": change            
        })
        
        if side == "buy":
            if amount == "0":
                del Bids[price]
            else:
                Bids[price] = float(amount)
        else:
            if amount == "0":
                del Asks[price]
            else:
                Asks[price] = float(amount)   

    return(starting_time, updates)

In [ ]:
def shortened_updates(file_name,K):
    starting_time, updates = parse_file_json(file_name)
    
    res = []
    
    # Caluclate first reference price
    starting_bids = updates[0]['Bids']
    starting_asks = updates[0]['Asks']
    sorted_bids = list(reversed(sorted(starting_bids.items())))
    sorted_asks = list(sorted(starting_asks.items()))        
    best_bid = sorted_bids[0][0]
    best_ask = sorted_asks[0][0]
    if ((best_bid + best_ask) % 2) != 0:
        old_reference_price = round((best_bid+best_ask)/2, 1)
    else:
        old_reference_price = round((best_bid+best_ask+1)/2, 1)
            
    for u in updates:      
        # Find reference price
        sorted_bids = list(reversed(sorted(u['Bids'].items())))
        sorted_asks = list(sorted(u['Asks'].items()))        
        best_bid = sorted_bids[0][0]
        best_ask = sorted_asks[0][0]
        if ((best_bid + best_ask) % 2) != 0:
            reference_price = round((best_bid+best_ask)/2, 1)
        else:
            middle = (best_bid+best_ask)/2
            if old_reference_price > middle:
                reference_price = round((best_bid+best_ask)/2 + 0.5,1)
            else:
                old_reference_price = round((best_bid+best_ask)/2 - 0.5,1)
                
        shortened_book = OrderedDict([(k,0) for k in range(-K,K+1) if k != 0])
        first_bid = int(round(reference_price - 0.5))
        first_ask = int(round(reference_price + 0.5))        
        for k in range(-K,0):
            shortened_book[k] = u['Bids'].get(first_bid + k + 1,0)
        for k in range(1,K+1):
            shortened_book[k] = u['Asks'].get(first_ask + k - 1,0)
            
        # Find k from the price. Keep track of event if
        # abs(k) <= K
        price = u["price"]
        k = price - reference_price
        if k < 0:
            k = int(round(k - 0.5))
        else:
            k = int(round(k + 0.5))
        if abs(k) <= K:                 
            res.append({
                'reference_price': reference_price,
                'LOB': copy.copy(shortened_book),
                'k': k,
                'change': u['change'],
                'time': u['time']
            })
        
        old_reference_price = reference_price

    return starting_time, res

starting_time,updates = shortened_updates('12_29_18_data.json',10)
len(updates)

In [ ]:
# Displays Order Book
def display_order_book(bids, asks, K, reference_price=None):
    if reference_price == None:
        sorted_bids = list(reversed(sorted(bids.items())))
        sorted_asks = list(sorted(asks.items()))        
        best_bid = sorted_bids[0][0]
        best_ask = sorted_asks[0][0]
        if ((best_bid + best_ask) % 2) != 0:
            reference_price = round((best_bid+best_ask)/2, 1)
        else:
            reference_price = round((best_bid+best_ask+1)/2, 1)
        
    bids = bids.items()
    bids = [b for b in bids if (reference_price - b[0]) < K]
    bid_prices = [b[0] for b in bids]
    bid_volumes = [b[1] for b in bids]

    asks = asks.items()
    asks = [a for a in asks if (a[0] - reference_price) < K]
    ask_prices = [a[0] for a in asks]
    ask_volumes = [a[1] for a in asks]

    b1 = plt.bar(bid_prices, bid_volumes, color='r')
    b2 = plt.bar(ask_prices, ask_volumes, color='g')
    v = plt.axvline(x=reference_price, color='b')
    plt.xticks([x for x in range(min(bid_prices), max(ask_prices)+1)])

    plt.title('ETC-USD Limit Order Book: Depth = {}'.format(K))
    plt.legend(['Reference Price = {}'.format(reference_price), 'Bids', 'Asks'])
    plt.xlabel('Price (USD Cents)')
    plt.ylabel('Volume')
    plt.show()

# Prints LOB
def print_order_book(bids, asks, K, reference_price=None):
    if reference_price == None:
        sorted_bids = list(reversed(sorted(bids.items())))
        sorted_asks = list(sorted(asks.items()))        
        best_bid = sorted_bids[0][0]
        best_ask = sorted_asks[0][0]
        if ((best_bid + best_ask) % 2) != 0:
            reference_price = round((best_bid+best_ask)/2, 1)
        else:
            reference_price = round((best_bid+best_ask+1)/2, 1)
        
    first_bid = int(round(reference_price - 0.5))
    first_ask = int(round(reference_price + 0.5))      
    
    print("Reference Price: {}".format(reference_price))
    print("---------------------------------------")

    print("First {} Bids: ".format(K))
    for price in range(first_bid - K + 1, first_bid + 1):
        print("{}, {}".format(price, bids.get(price, 0)))
    print("---------------------------------------")

    print("First {} Asks: ".format(K))
    for price in range(first_ask, first_ask + K):
        print("{}, {}".format(price, asks.get(price, 0)))
    print("---------------------------------------")
    print("\n")
    
display_order_book(parsed_file[1][0]['Bids'],parsed_file[1][0]['Asks'],10)
print_order_book(parsed_file[1][0]['Bids'],parsed_file[1][0]['Asks'],10)

In [ ]:
# Displays Order Book
def display_order_book_shortened(shortened_book, reference_price):    
    K = list(shortened_book.keys())[-1]
    bid_prices = [int(round(x[0] + reference_price + 0.5)) for x in shortened_book.items() if x[0] < 0]
    bid_volumes = [x[1] for x in shortened_book.items() if x[0] < 0]

    ask_prices = [int(round(x[0] + reference_price - 0.5)) for x in shortened_book.items() if x[0] > 0]
    ask_volumes = [x[1] for x in shortened_book.items() if x[0] > 0]

    b1 = plt.bar(bid_prices, bid_volumes, color='r')
    b2 = plt.bar(ask_prices, ask_volumes, color='g')
    v = plt.axvline(x=reference_price, color='b')
    plt.xticks([x for x in range(min(bid_prices), max(ask_prices)+1)])

    plt.title('ETC-USD Limit Order Book: Depth = {}'.format(K))
    plt.legend(['Reference Price = {}'.format(reference_price), 'Bids', 'Asks'])
    plt.xlabel('Price (USD Cents)')
    plt.ylabel('Volume')
    plt.show()
    
def print_order_book_shortened(shortened_book, reference_price):      
    first_bid = int(round(reference_price - 0.5))
    first_ask = int(round(reference_price + 0.5))  
    K = list(shortened_book.keys())[-1]

    print("Reference Price: {}".format(reference_price))
    print("---------------------------------------")

    print("First {} Bids: ".format(K))
    for k in range(-K, 0):
        price = int(round(k + reference_price + 0.5))
        print("{}, {}".format(price, shortened_book[k]))
    print("---------------------------------------")

    print("First {} Asks: ".format(K))
    for k in range(1, K+1):
        price = int(round(k + reference_price - 0.5))
        print("{}, {}".format(price, shortened_book[k]))
    print("---------------------------------------")
    print("\n")    
    
display_order_book_shortened(updates[0]['LOB'],updates[0]['reference_price'])
print_order_book_shortened(updates[0]['LOB'],updates[0]['reference_price'])

In [ ]:
updates[-1]['time']